In [1]:
from openai import OpenAI
import os

with open("/home/savitha07/.env") as env:
    for line in env:
        key, value = line.strip().split('=')
        os.environ[key] = value

client = OpenAI(
    api_key=os.environ.get('OPENAI_API_KEY'),
)

In [2]:
# ! pip install pandas

In [3]:
import pandas as pd

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_27120\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
n = 2000

In [7]:
df = pd.read_excel('Medicine_description.xlsx', sheet_name='Sheet1', 
        header=0, nrows=n)

In [6]:
# ! pip install openpyxl

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/250.0 kB 435.7 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/250.0 kB 939.4 kB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 1.7 MB/s eta 0:00:00


In [8]:
reasons = df["Reason"].unique()

In [9]:
reasons_dict = {reason: i for i, reason in enumerate(reasons)}

In [10]:
df["Drug_Name"] = "Drug: " + df["Drug_Name"] + "\n" + "Malady:"


In [11]:
df["Reason"] = " " + df["Reason"].apply(lambda x: "" + str(reasons_dict[x]))

In [12]:
df.drop(["Description"], axis=1, inplace=True)

In [13]:
df.rename(columns={"Drug_Name": "prompt", "Reason": "completion"}, inplace=True)

In [14]:
jsonl = df.to_json(orient="records", indent=0, lines=True)

In [15]:
with open("drug_malady_data.jsonl", "w") as f:
    f.write(jsonl)

In [17]:
# ! openai tools fine_tunes.prepare_data -f drug_malady_data.jsonl

client.files.create(
  file=open("drug_malady_data.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-E5W9Vm6hBsLebHVjfcT27do6', bytes=162256, created_at=1711427314, filename='drug_malady_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [18]:
client.fine_tuning.jobs.create(
  training_file="file-E5W9Vm6hBsLebHVjfcT27do6", 
  model="davinci-002"
)

FineTuningJob(id='ftjob-kOYDRkdC6hWOI1ArRPcwqANv', created_at=1711428186, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='davinci-002', object='fine_tuning.job', organization_id='org-U46Cfk9Rv4OSLkpbzpyC0snW', result_files=[], status='validating_files', trained_tokens=None, training_file='file-E5W9Vm6hBsLebHVjfcT27do6', validation_file=None, user_provided_suffix=None)

In [30]:
client.fine_tuning.jobs.retrieve("ftjob-kOYDRkdC6hWOI1ArRPcwqANv")

FineTuningJob(id='ftjob-kOYDRkdC6hWOI1ArRPcwqANv', created_at=1711428186, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:davinci-002:learninggpt::96tTuS2j', finished_at=1711430265, hyperparameters=Hyperparameters(n_epochs=3, batch_size=4, learning_rate_multiplier=2), model='davinci-002', object='fine_tuning.job', organization_id='org-U46Cfk9Rv4OSLkpbzpyC0snW', result_files=['file-BqydDrWbCZGKosQgcMoIynZk'], status='succeeded', trained_tokens=128130, training_file='file-E5W9Vm6hBsLebHVjfcT27do6', validation_file=None, user_provided_suffix=None)

In [61]:
model = 'ft:davinci-002:learninggpt::96tTuS2j'
drugs = [
    "A CN Gel(Topical) 20gmA CN Soap 75gm", 
    "Acolate M Kid Syrup 60ml",                    
    "ABICET M Tablet 10's",                                 
]

for drug_name in drugs:
    prompt = "Drug: {}\nMalady:".format(drug_name)

    response = client.completions.create(
        model=model,
        prompt=prompt,
        temperature=1,
        max_tokens=2
    )

    drug_class = response.choices[0].text.strip()

    print(drug_class)

0
2
2
